# Projeto de sistema de recomendação de filmes
## Passo 1) Lendo o dataset

O dataset utilizado é o MovieLens, cujo detalhamento pode ser encontrado no link http://files.grouplens.org/datasets/movielens/ml-20m-README.html

Os arquivos são disponibilizados no formato '.csv'. Assim, faremos a leitura dos arquivos com a biblioteca Pandas.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Lendo os arquivos
ratings = pd.read_csv('ml-20m/ratings.csv')
movies = pd.read_csv('ml-20m/movies.csv')
#tags = pd.read_csv('ml-20m/tags.csv')

In [3]:
# Mesclando os dados dos filmes com as avaliações
ratings_movies = pd.merge(ratings, movies, on='movieId').drop('timestamp', axis=1)
ratings_movies.head()

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,Jumanji (1995),Adventure|Children|Fantasy


In [4]:
# Retirando as linhas sem tag (NaN)
#tags = tags.dropna().drop('timestamp', axis=1)
#tags.head()

,userId,movieId,tag
0,18,4141,Mark Waters
1,65,208,dark hero
2,65,353,dark hero
3,65,521,noir thriller
4,65,592,dark hero


## Passo 2) Como explorar os dados?

Neste ponto a intenção é conhecer melhor os dados, explorando através de questões. Primeiro respondemos questões simples e mais genéricas, como os 10 filmes com mais avaliações 5 estrelas. Depois partimos para questões mais complexas relacionadas a preferência geral.

In [5]:
# Filtrar os 10 filmes com maior NÚMERO de avaliações 5 estrelas, listando-os pelo título:
top_5star_movies = ratings_movies[ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_movies

Shawshank Redemption, The (1994)             31896
Pulp Fiction (1994)                          27762
Silence of the Lambs, The (1991)             22513
Schindler's List (1993)                      22355
Star Wars: Episode IV - A New Hope (1977)    22117
Forrest Gump (1994)                          21292
Godfather, The (1972)                        20251
Usual Suspects, The (1995)                   19914
Matrix, The (1999)                           18582
Braveheart (1995)                            18467
Name: title, dtype: int64

In [6]:
# Os 25 filmes com maior MÉDIA de estrelas, listando-os pelo título:
top_meanstar = ratings_movies.groupby('title').agg({'rating': [np.size, np.mean]})
top_meanstar.sort_values([('rating', 'mean')], ascending=False).head(25)

rating     
                                                     size mean
title                                                         
Prom Queen: The Marc Hall Story (2004)                1.0  5.0
The Garden of Sinners - Chapter 5: Paradox Para...    1.0  5.0
Death of a Nation - The Timor Conspiracy (1994)       1.0  5.0
Poison (1951)                                         1.0  5.0
Sun Kissed (2012)                                     1.0  5.0
Giorgino (1994)                                       1.0  5.0
Schmatta: Rags to Riches to Rags (2009)               1.0  5.0
De la servitude moderne (2009)                        1.0  5.0
The Encounter (2010)                                  1.0  5.0
Best of Ernie and Bert, The (1988)                    1.0  5.0
Summer Wishes, Winter Dreams (1973)                   1.0  5.0
The Floating Castle (2012)                            1.0  5.0
Marihuana (1936)                                      1.0  5.0
Lady of Chance, A (1928)                              1.0  5.0
Such Hawks Such Hounds (2008)                         1.0  5.0
Tales That Witness Madness (1973)                     1.0  5.0
B-Side (2013)                                         1.0  5.0
Consuming Kids: The Commercialization of Childh...    2.0  5.0
Abendland (2011)                                      1.0  5.0
Going Down in LA-LA Land (2011)                       1.0  5.0
Mishen (Target) (2011)                                1.0  5.0
Argentina latente (2007)                              1.0  5.0
Flight of the Conchords: A Texan Odyssey (2006)       1.0  5.0
Always for Pleasure (1978)                            1.0  5.0
The Green (2011)                                      1.0  5.0

##### Diferença entre usar o número de avaliações 5 estrelas e a média de estrelas por filme:

Ao utilizar a número de avaliações 5 estrelas nós selecionamos os títulos mais populares e bem avaliados, entretanto, podem ficar subamostrados os bons títulos mas com poucas avaliações. Neste ponto supre tal necessidade o uso da média de estrelas por título, mas este último método também oferece a desvantagem de selecionar títulos que não são populares e também títulos que tiveram pouquissimas avaliações, contudo positivas.

### Questões genéricas relacionadas aos gêneros

* Quais os filmes com mais avaliações 5 estrelas dentro de cada gênero?

In [7]:
# Um exemplo para filtrar os filmes por gênero e por mais avaliações 5 estrelas, listando-os pelo título:
top_5star_drama = ratings_movies[ratings_movies['genres'].str.contains('Drama')][ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_drama

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Shawshank Redemption, The (1994)    31896
Pulp Fiction (1994)                 27762
Schindler's List (1993)             22355
Forrest Gump (1994)                 21292
Godfather, The (1972)               20251
Braveheart (1995)                   18467
American Beauty (1999)              15719
Fargo (1996)                        15232
Fight Club (1999)                   14623
Godfather: Part II, The (1974)      11737
Name: title, dtype: int64

### Questões relacionadas a preferência por gênero

Usuários avaliam de apenas um gênero, ou mais gêneros? Eles gostam desses gêneros?

Para responder a tal pergunta nós seguiremos as seguintes etapas: 

 1. Determinar o número de avaliações por gênero e usuário
 
 2. Determinar a participação de cada gênero no número de avaliações de cada usuário
 
 3. Encontrar quais os gêneros que compõe a maior parte das avaliações do usuário (determinar um threshold)
 
 4. Fazer a média do número de gêneros identificados por usuário como forma de metrificar a importância do gênero na escolha do filme.
 
O resultado mais próximo de 1 significa que os usuários avaliam em média somente um gênero; neste caso, identificar este gênero é muito importante para o sistema de recomendação. 
Já o resultado mais distante de 1 significa que os usuários avaliam em média muitos gêneros. 

In [8]:
# Determinando o número de gêneros avaliado por cada usuário

# 1. definindo todos os gêneros que existem:
genre_labels = set()
for s in ratings_movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

In [9]:
# 2. desmembrando todos os gêneros em colunas separadas
import re
genres_df = pd.DataFrame(dict((genre, ratings_movies.genres.str.contains(genre, re.IGNORECASE))
                             for genre in genre_labels))
ratings_movies_expand = genres_df.join(ratings_movies)
ratings_movies_expand.head()

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Romance,Sci-Fi,Thriller,War,Western,userId,movieId,rating,title,genres
0,False,False,True,False,True,False,False,False,False,True,...,False,False,False,False,False,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,False,False,True,False,True,False,False,False,False,True,...,False,False,False,False,False,5,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
2,False,False,True,False,True,False,False,False,False,True,...,False,False,False,False,False,13,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
3,False,False,True,False,True,False,False,False,False,True,...,False,False,False,False,False,29,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
4,False,False,True,False,True,False,False,False,False,True,...,False,False,False,False,False,34,2,3.0,Jumanji (1995),Adventure|Children|Fantasy


In [10]:
# 3. contar quantas vezes os gêneros são verdadeiros por usuário

n_gen_user = ratings_movies_expand.groupby('userId').agg({genre:[np.count_nonzero] for genre in genre_labels})
n_gen_user.head(10)

,Adventure,Crime,War,Fantasy,Musical,Children,Sci-Fi,Documentary,Thriller,Western,Drama,IMAX,Romance,Action,Mystery,Horror,Comedy,Animation,(no genres listed),Film-Noir
,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero,count_nonzero
userId,,,,,,,,,,,,,,,,,,,,
1,73,21,9,69,3,19,40,0,42,4,43,2,11,66,18,45,41,10,0,0
2,17,1,4,1,2,1,23,0,19,2,19,1,6,19,4,18,10,1,0,1
3,50,21,6,20,6,10,93,1,50,3,58,0,16,61,11,32,52,4,0,1
4,6,6,1,3,2,4,5,0,13,1,8,0,4,13,3,0,11,2,0,0
5,21,7,1,11,8,11,10,0,15,2,27,3,16,18,2,1,24,6,0,0
6,8,2,0,4,1,2,3,0,6,0,6,0,9,7,2,0,12,1,0,0
7,59,17,17,23,9,17,68,0,45,9,130,1,112,61,16,11,122,7,0,4
8,19,13,5,7,5,7,11,0,26,3,26,3,21,30,3,3,25,7,0,0


In [23]:
#  Determinando a participação de cada gênero no número de avaliações de cada usuário

n_gen_user['total'] = n_gen_user.apply(sum, axis=1)

nan_gen_user = n_gen_user.replace(0,'NaN')

#tax_gen_user = nan_gen_user.div(nan_gen_user['total'])

# Guia dos próximos passos:


1. Verificar as preferências do usuário por gênero, ele gosta de um ou mais gêneros? Identificar um o mais gêneros(tentar colocar variável)

2. Filtrar os filmes que tem esses gêneros

3. Excluir o que o usuário já assistiu

4. Rankear pelas avaliações (todas as avaliações ou somente do pessoal que tem as mesmas preferências?)

5. Sugerir ao usuário os x melhores no ranking